In [2]:
import pandas as pd

In [3]:
address_df = pd.read_csv("data/address_eth.csv")
transaction_df = pd.read_csv("data/tx_eth.csv")

In [9]:
a = address_df[~address_df["address"].isin(transaction_df["from"])]



address_df["address"].isin(transaction_df["to"])

0         True
1         True
2         True
3         True
4         True
         ...  
16652    False
16653    False
16654    False
16655    False
16656    False
Name: address, Length: 16657, dtype: bool